In [22]:
from disk_models import *

x0 = np.pi*np.array([1, 1, -1.5, 1, -1, 1, 0, 0]).reshape(-1, 1)
u0 = np.pi*np.array([-1, 0.2]).reshape(-1, 1)
dynamics_model = DynamicsModel('model')    

print(dynamics_model(x0, u0))
print(model(x0, u0))

dynamics_model_with_forward = DynamicsModelWithForward('model_with_forward')
x = ca.MX.sym("x",8,1)
u = ca.MX.sym("x",2,1)
    
#f = ca.Function('f',[x,u],[model(x,u)])
J0 = ca.Function('J0',[x,u],[ca.jacobian(model(x,u),x)])
print('evaluate function call for casadi function')
%timeit J0(x0,u0)

J1 = ca.Function('J1',[x,u],[ca.jacobian(dynamics_model_with_forward(x,u),x)])
print('evaluate function call for casadi call function with forward')
%timeit J1(x0,u0)

x0 = ca.pi*ca.DM([[1, 1, -1.5, 1, -1, 1, 0, 0],[1, 1, -1.5, 1, -1, 1, 0, 0]]).T
u0 = ca.pi*ca.DM([[-1, 0.2],[-5, 0.3]]).T
%timeit J0(x0,u0)
%timeit J1(x0,u0)

initializing object
[3.14159, -3.14159, 3.14159, -38.6039, -105.347, 165.133, -314.159, 62.8319]
[3.14159, -3.14159, 3.14159, -38.6039, -105.347, 165.133, -314.159, 62.8319]
initializing object
evaluate function call for casadi function
16.6 µs ± 496 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
evaluate function call for casadi call function with forward
307 µs ± 882 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
7.84 µs ± 49.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
573 µs ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [21]:
horizon = 20
nx = 8
nu = 2
dt = 0.05
x0 = np.pi*np.array([1, 1, -1.5, 1, -1, 1, 0, 0]).reshape(-1,1)

option = {}
option['max_iter']=10000

#option['print_level']=0

opti = ca.Opti()
x = opti.variable(nx,horizon+1)
u = opti.variable(nu,horizon)

phi_1= x[0,:]
phi_2= x[1,:]
phi_3= x[2,:]
dphi_1= x[3,:]
dphi_2= x[4,:]
dphi_3= x[5,:]
phi_1_m= x[6,:]
phi_2_m= x[7,:]

phi_m_1_set = u[0,:]
phi_m_2_set = u[1,:]

model=DynamicsModelWithForward('m')    

k1 = model(x[:,0:-1],u)
k2 = model(x[:,0:-1]+dt/2*k1,u)
k3 = model(x[:,0:-1]+dt/2*k2,u)
k4 = model(x[:,0:-1]+dt*k3,u)

opti.minimize(ca.dot(phi_1,phi_1)+ca.dot(phi_2,phi_2)+ca.dot(phi_3,phi_3)
              +0.001*ca.dot(phi_m_1_set,phi_m_1_set)+0.1*ca.dot(phi_m_2_set,phi_m_2_set)
              +0.001*ca.dot(phi_1_m,phi_1_m)+0.1*ca.dot(phi_2_m,phi_2_m))

opti.subject_to(x[:,1:] == x[:,0:-1]+dt/6*(k1+2*k2+2*k3+k4))

opti.subject_to(x[:,0]==x0)

opti.subject_to(opti.bounded(-2*ca.pi,phi_1,2*ca.pi))
opti.subject_to(opti.bounded(-2*ca.pi,phi_2,2*ca.pi))
opti.subject_to(opti.bounded(-2*ca.pi,phi_3,2*ca.pi))

opti.subject_to(opti.bounded(-2*ca.pi,phi_1_m,2*ca.pi))
opti.subject_to(opti.bounded(-2*ca.pi,phi_1_m,2*ca.pi))


opti.subject_to(opti.bounded(-2*ca.pi,phi_m_1_set,2*ca.pi))
opti.subject_to(opti.bounded(-2*ca.pi,phi_m_2_set,2*ca.pi))



opti.solver("ipopt",{},option)

try:
    sol = opti.solve()

except Exception as e:
    print(e)

initializing object
Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/function_internal.cpp:145: Error calling IpoptInterface::init for 'solver':
Error in Function::factory for 'nlp' [MXFunction] at .../casadi/core/function.cpp:1634:
Failed to create nlp_grad:[x, p, lam:f, lam:g]->[f, g, grad:gamma:x, grad:gamma:p] with {"gamma": [f, g]}:
.../casadi/core/factory.hpp:367: Gradient generation failed:
Error in MX::gradient at .../casadi/core/generic_matrix.hpp:1203:
Error in MX::jtimes at .../casadi/core/generic_matrix.hpp:1191:
Error in MX::reverse at .../casadi/core/mx.cpp:1741:
Error in MXFunction::ad_reverse at .../casadi/core/mx_function.cpp:1042:
Error in MX::ad_reverse for node of type N6casadi4CallE at .../casadi/core/mx.cpp:2044:
Error in Call::ad_reverse for 'm' [CallbackInternal] at .../casadi/core/casadi_call.cpp:147:
Error in Function::jacobian for 'wrap_m' [MXFunction] at .../casadi/core/function.cpp:824:
Error in XFunction::get_jacobian f

In [26]:
import tensorflow as tf
@tf.function
def add(a,b):
    return a+b


w = tf.Variable([[1.], [2.]])
v = tf.Variable([[1.], [2.]])
z=w+v
print(z)

z(tf.constant([[3.], [4.]]),tf.constant([[3.], [4.]]))


tf.Tensor(
[[2.]
 [4.]], shape=(2, 1), dtype=float32)


TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable